In [1]:
import findspark
findspark.init('/home/fede/spark-2.1.0-bin-hadoop2.7')

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [3]:
file_path = '/home/fede/seeds_dataset.csv'
dataset = spark.read.csv(file_path, header=True,inferSchema=True)

In [4]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [5]:
dataset.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [6]:
from pyspark.ml.clustering import KMeans

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [9]:
assembler = VectorAssembler(inputCols = dataset.columns,
                           outputCol='features')

In [10]:
final_data = assembler.transform(dataset)

In [11]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
from pyspark.ml.feature import StandardScaler

In [13]:
scaler = StandardScaler(inputCol='features',
                       outputCol = 'scaledFeatures')

In [14]:
scaler_model = scaler.fit(final_data)

In [15]:
final_data = scaler_model.transform(final_data)

In [16]:
final_data.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [19]:
kmeans = KMeans(featuresCol='scaledFeatures', k=3)

In [20]:
model=kmeans.fit(final_data)

In [21]:
print('WSSSE')
print(model.computeCost(final_data))

WSSSE
428.76501177873223


In [22]:
centers = model.clusterCenters()
print(centers)

[array([  4.06660859,  10.14191893,  35.84098009,  11.81592066,
         7.52397236,   3.1823335 ,  10.39801233]), array([  6.3407095 ,  12.39263108,  37.41143125,  13.92892299,
         9.77251635,   2.42396447,  12.28547936]), array([  4.9290032 ,  10.94057945,  37.3117462 ,  12.40145129,
         8.60300551,   1.8135783 ,  10.38202259])]


In [23]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         1|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         0|
+----------+
only showing top 20 rows

